In [8]:
import torch.nn as nn
import cv2
from torchvision import transforms
import torch
import torchvision
import numpy as np
import ds_ear
import glob
from PIL import Image
from matplotlib import image
from DLBio import pt_training
from os.path import join



CATEGORIES = ["Falco","Konrad", "Jesse", "Nils"]
AUTHENTIFICATED = ["Falco","Konrad"]
RESIZE_Y = 150
RESIZE_X = 100
DATA_TEST_FOLDER = "../test/*png"


def get_data(folder):
    img_array = []
    img_array_resized = []
    files = glob.glob (folder)
    for idx, f in zip(range(len(files)),files):
        image = cv2.imread(f)
        img_array.append (image)
        img_array_resized.append(cv2.resize(img_array[idx],(RESIZE_Y,RESIZE_X)))
    return np.asarray(img_array_resized)


model = torch.load('./class_sample/model.pt')



In [9]:
image_array = []
files = glob.glob (DATA_TEST_FOLDER)
files.sort()
for idx, f in zip(range(len(files)),files):
    image = Image.open(f)
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize((RESIZE_Y, RESIZE_X)),
        torchvision.transforms.Lambda(lambda x: x.convert('RGB')),
        torchvision.transforms.ToTensor(),

        torchvision.transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225]
        )
    ])
    image_transformed = transform(image)
    image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    image_array.append(image_transformed.type('torch.cuda.FloatTensor'))


In [10]:
NUMBER_AUTHORIZED = int(.7*len(image_array))

all_classes = []

for i in image_array:
	with torch.no_grad():
		pred = model(i)
		pred = torch.softmax(pred, 1)
		pred = pred.cpu().numpy()

	classes = np.argmax(pred, 1)
	all_classes.append(classes[0])

	pred = np.append(pred, classes)
	pred = np.append(pred, CATEGORIES[classes[0]])	
	print(pred, "\n")
print(all_classes)


['0.7543401718139648' '0.24565979838371277' '0.0' 'Falco'] 

['0.6644101738929749' '0.33558985590934753' '0.0' 'Falco'] 

['0.6446036696434021' '0.3553963303565979' '0.0' 'Falco'] 

['0.6736924052238464' '0.3263075351715088' '0.0' 'Falco'] 

['0.7996309995651245' '0.20036901533603668' '0.0' 'Falco'] 

['0.7294602990150452' '0.27053970098495483' '0.0' 'Falco'] 

['0.7184112668037415' '0.28158876299858093' '0.0' 'Falco'] 

['0.798162043094635' '0.20183798670768738' '0.0' 'Falco'] 

['0.07654973119497299' '0.923450231552124' '1.0' 'Konrad'] 

['0.11979769170284271' '0.8802022933959961' '1.0' 'Konrad'] 

['0.0983697846531868' '0.9016302227973938' '1.0' 'Konrad'] 

['0.13855870068073273' '0.8614412546157837' '1.0' 'Konrad'] 

['0.22020338475704193' '0.7797966599464417' '1.0' 'Konrad'] 

['0.3235251307487488' '0.6764748692512512' '1.0' 'Konrad'] 

['0.3524317145347595' '0.6475682258605957' '1.0' 'Konrad'] 

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [11]:
authentification_dict = {CATEGORIES[i]:all_classes.count(i) for i in all_classes}
print(authentification_dict) 

authentification_dict.get()
# if np.max(counts) > NUMBER_AUTHORIZED:
# 	print("Welcome to your save room " + CATEGORIES[np.argmax(counts)] + "!")
# else: 
# 	print("Authentification Failed! You got no acces rights!")

{'Falco': 8, 'Konrad': 7}


In [ ]:
# TODO: OLD STUFF
# NUMBER_AUTHORIZED = int(.7*len(data_tensor))
# with torch.no_grad():
# 	pred = model(data_tensor)
# 	pred = torch.softmax(pred, 1)
# 	pred = pred.cpu().numpy()

# classes_ = np.argmax(pred, 1)
# print(pred)
# print(classes_)
# counts = np.bincount(classes_)
# if np.max(counts) > NUMBER_AUTHORIZED:
# 	print("Welcome to your save room " + CATEGORIES[np.argmax(counts)] + "!")
# else: 
# 	print("Authentification Failed! You got no acces rights!")


# data = get_data(DATA_TEST_FOLDER)
# data_tensor = torch.from_numpy(data)
# data_tensor = data_tensor.permute(0, 3, 1, 2)
# data_tensor = data_tensor.type('torch.cuda.FloatTensor')
#device = get_device()
#print(device)
#if device == cpu:
#    data_tensor = data_tensor.type('torch.FloatTensor')
#else:
#    data_tensor = data_tensor.type('torch.cuda.FloatTensor')


# 	print("Authentification Failed! You got no acces rights!")